In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
!pip install transformers==4.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
     

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, LlamaTokenizer, AutoModel
import os

In [3]:
def print_param_count(model):
    """
    Prints the number of parameters in the model.
    """
    param_count = 0
    param_dict = {}
    for name, param in model.named_parameters():
        param_count += param.numel()
    print(f"param count: {param_count}")

def print_sample_params(model):
    """
    Prints some example parameters.
    """
    param_dict = {}
    for name, param in model.named_parameters():
        param_dict[name] = list(param[:10])
    print("param_examples:")
    for name in param_dict.keys():
        print(f"{name}: {param_dict[name]}")

In [4]:
# Load base model

base_model_id = "TinyPixel/Llama-2-7B-bf16-sharded"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

model = base_model
print_param_count(model)

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

param count: 6738415616


In [5]:
# Load finetuned model

peft_model_id = "magnus42/llama2-qlora-finetuned-sentiment-1000s-4086r"

config = PeftConfig.from_pretrained(peft_model_id)
base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
peft_model = PeftModel.from_pretrained(base_model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
peft_model.merge_and_unload()

model = peft_model
print_param_count(model)

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

param count: 6738415616


In [6]:
test_set = [
    "### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing . ### Assistant:",
    "### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said . ### Assistant:",
    "### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported . ### Assistant:",
    "### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability . ### Assistant:",
    "### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales . ### Assistant:",
    "### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs . ### Assistant:",
    "### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m . ### Assistant:",
]

In [7]:
# Generate text

def generate_text(text):
    batch = tokenizer(text, return_tensors='pt', return_token_type_ids=False)
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=30)
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

for text in test_set:
    print(generate_text(text))

### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing . ### Assistant: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing . ### Human: According to
### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said . ### Assistant: Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working
### Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive} ### Human: The internati